In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams-ionization/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot
import plot
import fluxprof

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'

# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS

# Read in Snapshots

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    # shot[ii].load_shock_variables()
    snaps[ii].load_fluxes()

draw_flux_vs_height()

In [ ]:
def draw_flux_vs_height(ax, xx, yy, labels, fmt, alpha=0.6, colors=lightcolors):
    # colors = cm.gist_rainbow_r(np.linspace(0, 1, len(snaps)))
    for ii in range(len(yy)):
        ax.plot(xx, yy[ii], fmt, color=colors[ii], label = (labels[ii]), alpha=alpha)



# All-Res

In [ ]:
rnums = np.array([9, 21, 39, 54])
ryears = years[1:len(rnums)+1]
rmasss = masss[1:len(rnums)+1]
snaps_lr, snaps_mr, snaps_hr = fluxprof.prep_res_snapshots()

mass_lr = np.array(['2 M$_\odot$ lr', '4 M$_\odot$ lr', '8 M$_\odot$ lr', '12 M$_\odot$ lr',])
mass_mr = np.array(['2 M$_\odot$ mr', '4 M$_\odot$ mr', '8 M$_\odot$ mr', '12 M$_\odot$ mr',])
mass_hr = np.array(['2 M$_\odot$ hr', '4 M$_\odot$ hr', '8 M$_\odot$ hr', '12 M$_\odot$ hr',])
# print(mass_lr, mass_mr, mass_hr)

## Res Z Segments

In [ ]:
debug = False
targets=np.linspace(0,26000,27)
if debug: print(targets)

zseg_lr, zval_lr, zmid_lr = fluxprof.get_z_segments(snaps_lr[0].x1, targets)
if debug: print(f"{zseg_lr=} \n{zval_lr=} \n{zmid_lr=}")

zseg_mr, zval_mr, zmid_mr = fluxprof.get_z_segments(snaps_mr[0].x1, targets)
if debug: print(f"{zseg_mr=} \n{zval_mr=} \n{zmid_mr=}")

zseg_hr, zval_hr, zmid_hr = fluxprof.get_z_segments(snaps_hr[0].x1, targets)
if debug: print(f"{zseg_hr=} \n{zval_hr=} \n{zmid_hr=}")

## Calculate Flux Profiles

### Without scaling

In [ ]:
NORM = False
norm_label = 'scaled to 1000 au' if NORM else 'unscaled pixel regions'

for ratio in [True, False]:
    for ss, snap in enumerate(snaps_lr):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_lr, ratio=ratio, debug=False, norm=NORM)

    for ss, snap in enumerate(snaps_mr):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_mr, ratio=ratio, debug=False, norm=NORM)

    for ss, snap in enumerate(snaps_hr):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_hr, ratio=ratio, debug=False, norm=NORM)

## Plot Flux vs Height AllRes

In [ ]:
def plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr, xx_mr, xx_hr, 
        mass_labels,  
        ff_arr = [5,9],
        ymin0 = None, ymax0 = None, ymin1 = None, ymax1 = None, 
        xscale='log', ax0_text = None, ax1_text=None, xx_text = 0.02, yy_text = 0.02, 
        leg_anchor = (1, 0)
):
    fig, axs = plot.figax(sharex=True,
                         xscale=xscale, nrows=2, ncols=2, figsize=(11,6))

    all_snaps = np.array([snaps_lr, snaps_mr, snaps_hr])
    all_xx = np.array([xx_lr, xx_mr, xx_hr])
    all_fmt = np.array([':x', '--+', '-o',])
    none_labels = np.repeat('', len(snaps_lr))

    # first row: no ratio
    for ii, ax in enumerate(axs[0,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[jj]
            if ii==0 and jj==0: labels = mass_labels
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt)

        ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ax.set_ylabel(ylabel)
        if ax0_text is not None:
            ax.text(xx_text, yy_text, ax0_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')
            
    if ymin0 is not None and ymax0 is not None:
        axs[0,0].set_ylim(ymin0, ymax0)

    axs[0,1].sharey(axs[0,0])

    # second row: ratio
    for ii, ax in enumerate(axs[1,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[jj]
            if ii==1 and jj==2: labels = mass_labels
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof_ratio
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt)

        ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ax.set_ylabel(ylabel)
        ax.set_xlabel('$z$ (au)')
        if ax1_text is not None:
            ax.text(xx_text, yy_text, ax1_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')

    if ymin1 is not None and ymax1 is not None:
        axs[1,0].set_ylim(ymin1, ymax1)

    axs[1,1].sharey(axs[1,0])
    axs[1,1].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')

    fig.tight_layout()
    return fig
    

In [ ]:

fig = plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr=zmid_lr[:-1], xx_mr=zmid_mr[:-1], xx_hr=zmid_hr[:-1], 
        mass_labels=masss[1:len(rnums)+1],
        xscale='linear', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-10, ymax0 = 10**2.5, ymin1 = 10**-10, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.08))
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()
# fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/allres_fluxprofile_norm_lin.png', dpi=300)

In [ ]:
fig = plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr=zmid_lr[:-1], xx_mr=zmid_mr[:-1], xx_hr=zmid_hr[:-1], 
        mass_labels=masss[1:len(rnums)+1],
        xscale='log', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-10, ymax0 = 10**2.5, ymin1 = 10**-10, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1.01, 0.05))
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()
# fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/allres_fluxprofile_norm_log.png', dpi=300)


### With Scaling

In [ ]:
NORM = True
norm_label = 'scaled to 1000 au' if NORM else 'unscaled pixel regions'

for ratio in [True, False]:
    for ss, snap in enumerate(snaps_lr):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_lr, ratio=ratio, debug=False, norm=NORM)

    for ss, snap in enumerate(snaps_mr):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_mr, ratio=ratio, debug=False, norm=NORM)

    for ss, snap in enumerate(snaps_hr):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_hr, ratio=ratio, debug=False, norm=NORM)

In [ ]:

fig = plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr=zmid_lr[:-1], xx_mr=zmid_mr[:-1], xx_hr=zmid_hr[:-1], 
        mass_labels=masss[1:len(rnums)+1],
        xscale='linear', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-10, ymax0 = 10**2.5, ymin1 = 10**-10, ymax1 = 10**-1, 
        xx_text=0.77, yy_text=0.02, 
        leg_anchor=(1, 0.08))
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/allres_fluxprofile_lin.png', dpi=300)
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()
# fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/allres_fluxprofile_norm_lin.png', dpi=300)

In [ ]:
fig = plot_res_flux_vs_height( 
        snaps_lr, snaps_mr, snaps_hr, 
        xx_lr=zmid_lr[:-1], xx_mr=zmid_mr[:-1], xx_hr=zmid_hr[:-1], 
        mass_labels=masss[1:len(rnums)+1],
        xscale='log', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-10, ymax0 = 10**2.5, ymin1 = 10**-10, ymax1 = 10**-1, 
        xx_text=0.77, yy_text=0.02, 
        leg_anchor=(1.01, 0.05))
fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/allres_fluxprofile_log.png', dpi=300)
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()

# 3 Mid-Res Only, New Code

In [ ]:
for ii, snap in enumerate(snaps):
    snap.calculate_deltas()

In [ ]:
NORM = True
norm_label = 'scaled to 1000 au' if NORM else 'unscaled pixel regions'


for ratio in [True, False]:
    for ss, snap in enumerate(snaps):
        print(snap.name, 'ratio=',ratio)
        fluxprof.make_flux_profiles(snap, zseg_mr, ratio=ratio, debug=False, norm=NORM)

## Plot mid res flux vs height all

In [ ]:
def plot_mid_flux_vs_height( 
        snaps_mr,  
        xx_mr, 
        mass_labels,  
        ff_arr = [5,9],
        ymin0 = None, ymax0 = None, ymin1 = None, ymax1 = None, 
        xscale='log', ax0_text = None, ax1_text=None, xx_text = 0.02, yy_text = 0.02, 
        leg_anchor = (1, 0),
        colors=lightcolors,
):
    fig, axs = plot.figax(sharex=True,
                         xscale=xscale, nrows=2, ncols=2, figsize=(11,6))

    all_snaps = np.array([snaps_mr, ])
    all_xx = np.array([ xx_mr, ])
    all_fmt = np.array(['-o',])
    # all_labels = np.array([labels_mr,])
    none_labels = np.repeat('', len(snaps_mr))

    # first row: no ratio
    for ii, ax in enumerate(axs[0,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[jj]
            if ii==0 and jj==0: labels = mass_labels
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors)

        ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ax.set_ylabel(ylabel)
        if ax0_text is not None:
            ax.text(xx_text, yy_text, ax0_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')
    if ymin0 is not None and ymax0 is not None:
        axs[0,0].set_ylim(ymin0, ymax0)
    axs[0,1].sharey(axs[0,0])

    # second row: ratio
    for ii, ax in enumerate(axs[1,:]):
        ff = ff_arr[ii]

        for jj, snaps in enumerate(all_snaps):
            xx = all_xx[jj]
            fmt = all_fmt[jj]
            if ii==1 and jj==0: 
                labels = mass_labels
                print('both 0')
            else: labels = none_labels

            yy = np.zeros((len(snaps), len(xx)))
            for ss, snap in enumerate(snaps):
                temp = snap.flux_prof_ratio
                yy[ss] = temp[ff,:-1]
            draw_flux_vs_height(ax, xx, yy, labels=labels, fmt=fmt, colors=colors)

        ylabel = r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[ff]/10**9)
        ax.set_ylabel(ylabel)
        ax.set_xlabel('$z$ (au)')
        if ax1_text is not None:
            ax.text(xx_text, yy_text, ax1_text, transform=ax.transAxes, 
                    weight='bold', horizontalalignment='right')

    if ymin1 is not None and ymax1 is not None:
        axs[1,0].set_ylim(ymin1, ymax1)
    axs[1,1].sharey(axs[1,0])
    axs[1,1].legend(bbox_to_anchor = leg_anchor, bbox_transform=ax.transAxes, loc='lower right')

    fig.tight_layout()
    return fig
    

In [ ]:

colors = np.array(['#ff3333', '#ff8c1a', '#bfbd2e',  '#5cd699', '#51a6fb', '#a65eed'])


fig = plot_mid_flux_vs_height( 
        snaps_mr=snaps[1:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[1:],
        xscale='linear', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-10, ymax0 = 10**2.5, ymin1 = 10**-10, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.08), colors=colors)
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()
# fig.savefig('/Users/emigardiner/VICO/pjams-ionization/figures/allres_fluxprofile_norm_lin.png', dpi=300)

In [ ]:

colors = np.array(['#ff3333', '#ff8c1a', '#bfbd2e',  '#5cd699', '#0d80f2', '#a65eed'])

fig = plot_mid_flux_vs_height( 
        snaps_mr=snaps[1:], xx_mr=zmid_mr[:-1], 
        mass_labels=masss[1:],
        xscale='log', ax0_text='no cooling', ax1_text='with cooling',
        ymin0 = 10**-10, ymax0 = 10**2.5, ymin1 = 10**-10, ymax1 = 10**-1, 
        xx_text=0.99, yy_text=0.025, 
        leg_anchor=(1, 0.08), colors=colors)
fig.text(0,0, norm_label, fontsize=14)
fig.tight_layout()
